In [15]:
import xarray as xr
import numpy as np
import xoak
from matplotlib import pyplot as plt
from cmocean import cm # for oceanography-specific colormaps

In [10]:
ds_mesh = xr.open_dataset("../../FESOM_data/channel/fesom.mesh.diag.nc")

channel_lon_bds = (0,4.5) # use inmutable objects
channel_lat_bds = (0,18)
number_lon = 2*72 
number_lat = 2*292

ds_mesh = ds_mesh.assign_coords(
    nod2=list(range(1, ds_mesh.sizes["nod2"]+1)), 
    elem=list(range(1,ds_mesh.sizes['elem']+1)),
)

ds_mesh

<xarray.Dataset> Size: 9MB
Dimensions:            (nz: 41, nz1: 40, elem: 21120, nod2: 10608, n3: 3,
                        n2: 2, edg_n: 31728, N: 6, n4: 4)
Coordinates:
  * nz                 (nz) float64 328B 0.0 9.038 18.98 ... 3.628e+03 4e+03
  * nz1                (nz1) float64 320B 4.519 14.01 ... 3.459e+03 3.814e+03
  * nod2               (nod2) int64 85kB 1 2 3 4 5 ... 10605 10606 10607 10608
  * elem               (elem) int64 169kB 1 2 3 4 5 ... 21117 21118 21119 21120
Dimensions without coordinates: n3, n2, edg_n, N, n4
Data variables: (12/21)
    fesom_mesh         int32 4B ...
    elem_area          (elem) float64 169kB ...
    nlevels_nod2D      (nod2) int32 42kB ...
    nlevels            (elem) int32 84kB ...
    nod_in_elem2D_num  (nod2) int32 42kB ...
    nod_part           (nod2) int32 42kB ...
    ...                 ...
    face_links         (n3, elem) float64 507kB ...
    edge_face_links    (n2, edg_n) float64 508kB ...
    nod_in_elem2D      (N, nod2) int32 255kB ...
    edge_cross_dxdy    (n4, edg_n) float64 1MB ...
    gradient_sca_x     (n3, elem) float64 507kB ...
    gradient_sca_y     (n3, elem) float64 507kB ...
Attributes:
    Conventions:  UGRID-1.0

In [3]:
ds_w = xr.open_mfdataset('../../FESOM_data/channel/w.fesom.2005.nc',
                         chunks = {'time':1, 'nz':1}).isel(time=slice(0,0))
ds_w

/opt/conda/lib/python3.11/site-packages/xarray/core/dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "nz" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.Dataset> Size: 328B
Dimensions:  (nz: 41, time: 0, nod2: 10608)
Coordinates:
  * nz       (nz) float64 328B 0.0 9.038 18.98 ... 3.29e+03 3.628e+03 4e+03
  * time     (time) datetime64[ns] 0B 
Dimensions without coordinates: nod2
Data variables:
    w        (time, nz, nod2) float32 0B dask.array<chunksize=(0, 1, 10608), meta=np.ndarray>
Attributes: (12/18)
    Conventions:                         UGRID-1.0
    FESOM_model:                         FESOM2
    FESOM_website:                       fesom.de
    FESOM_git_SHA:                       ab6d892b
    FESOM_MeshPath:                      /albedo/work/projects/p_clidyn_work/...
    FESOM_mesh_representative_checksum:  
    ...                                  ...
    FESOM_use_floatice:                  0
    FESOM_whichEVP:                      0
    FESOM_evp_rheol_steps:               120
    FESOM_opt_visc:                      5
    FESOM_use_wsplit:                    0
    FESOM_autorotate_back_to_geo:        0

In [6]:
grid_lon = xr.DataArray(np.linspace(*channel_lon_bds,number_lon), 
                        dims=('grid_lon',))
grid_lat = xr.DataArray(np.linspace(*channel_lat_bds,number_lat),
                        dims=('grid_lat',))


In [17]:
ds_mesh = ds_mesh.assign_coords(
    lat=("nod2", ds_mesh.lat.data.flatten()),
    lon=("nod2", ds_mesh.lon.data.flatten()),
)

ds_mesh.xoak.set_index(["lat", "lon"], "sklearn_geo_balltree")
ds_mesh
#
ds_mesh

<xarray.Dataset> Size: 9MB
Dimensions:            (nz: 41, nz1: 40, elem: 21120, nod2: 10608, n3: 3,
                        n2: 2, edg_n: 31728, N: 6, n4: 4)
Coordinates:
  * nz                 (nz) float64 328B 0.0 9.038 18.98 ... 3.628e+03 4e+03
  * nz1                (nz1) float64 320B 4.519 14.01 ... 3.459e+03 3.814e+03
    lon                (nod2) float64 85kB 0.0 0.0469 0.0 ... 4.406 4.453 4.406
    lat                (nod2) float64 85kB 0.0 0.0814 0.1628 ... 17.82 17.9
  * nod2               (nod2) int64 85kB 1 2 3 4 5 ... 10605 10606 10607 10608
  * elem               (elem) int64 169kB 1 2 3 4 5 ... 21117 21118 21119 21120
Dimensions without coordinates: n3, n2, edg_n, N, n4
Data variables: (12/19)
    fesom_mesh         int32 4B ...
    elem_area          (elem) float64 169kB ...
    nlevels_nod2D      (nod2) int32 42kB ...
    nlevels            (elem) int32 84kB ...
    nod_in_elem2D_num  (nod2) int32 42kB ...
    nod_part           (nod2) int32 42kB ...
    ...                 ...
    face_links         (n3, elem) float64 507kB ...
    edge_face_links    (n2, edg_n) float64 508kB ...
    nod_in_elem2D      (N, nod2) int32 255kB ...
    edge_cross_dxdy    (n4, edg_n) float64 1MB ...
    gradient_sca_x     (n3, elem) float64 507kB ...
    gradient_sca_y     (n3, elem) float64 507kB ...
Attributes:
    Conventions:  UGRID-1.0

In [29]:
target_lon, target_lat = xr.broadcast(grid_lon, grid_lat)
target_lon.shape


(144, 584)

In [ ]:
# grid_nodes = ds_mesh.xoak.sel(
#     lat = target_lat,
#     lon = target_lon,
# ).nod2

In [31]:
node2_grid = ds_mesh.xoak.sel(lon = target_lon,lat=target_lat).nod2
node2_grid

<xarray.DataArray 'nod2' (grid_lon: 144, grid_lat: 584)> Size: 673kB
array([[    1,     1,     2, ...,   221,   221,   221],
       [    1,     1,     2, ...,   221,   221,   221],
       [  222,   222,     2, ...,   442,   442,   442],
       ...,
       [10388, 10388, 10389, ..., 10608, 10608, 10608],
       [10388, 10389, 10389, ..., 10608, 10608, 10608],
       [10388, 10389, 10389, ..., 10608, 10608, 10608]])
Coordinates:
    lon      (grid_lon, grid_lat) float64 673kB 0.0 0.0 0.0469 ... 4.406 4.406
    lat      (grid_lon, grid_lat) float64 673kB 0.0 0.0 0.0814 ... 17.9 17.9
    nod2     (grid_lon, grid_lat) int64 673kB 1 1 2 2 ... 10608 10608 10608
Dimensions without coordinates: grid_lon, grid_lat

In [ ]:
W_grid = ds.W.isel(nod2 = node2_grid -1 ).isel(nz1 = nz1_grid -1). 

In [ ]:
""" Idea
    lat_grid, lon_grid
1)    node2_grid = ds_mesh.xoak.sel(lon=lon_grid,lat=lat_grid).nod2

2) z_grid
    nz1_grid = ds_mesh.sel(z1 = z_grid, method = 'nearest').nz1

3) ds.W.isel(nod2 = node2_grid -1 ).isel(nz1 = nz1_grid -1). 
"""
